# Zusammenfassungen und Erzählungen


## VL1: Raw Rates, methods (YoB, YoD, MortCoeff, Exposure)
Beginnen wir eine kurze Zusammenfassung der Vorlesung.
In der ersten Vorlesung wurde erstmals das Konzept der Sterberaten eingeführt als ANzahl der Tode durch Anzahl der Lebenden. Für die Berechnung der Sterberaten wurden verschiedene Konzepte und Schätzer eingeführt. 
* Die erste davon ist die Geburtsjahr-Methode, welche die Toten aus demselben Geburtsjahr nimmt. Diese können entweder im aktuellen oder aber im nächsten Jahr sterben, wenn sie noch vor ihrem Geburtstag sterben, der sie in die nächste Altersklasse wirft. 
* Die zweite ist die Sterbejahrmethode: Hier kommt es darauf an, wie alt man im effektiven Sterbejahr ist, egal ob man im einen oder anderen Jahr geboren wurde. 
* Schließlich gibt es noch einen Schätzer mit dem "Mortality coefficient", der ebenfalls auf das Sterbejahr setzt, aber ein wenig eine andere Methodik benutzt und damit erstmalig auch etwas über Migrationseffekte nachdenkt.
* Die Exposure-Methode entspricht im Wesentlichen einer Risk-pro-Rata Idee und flexibilisiert den Nenner etwas hin zu einer realistischeren Abschätzung der Anzahl der lebendigen. Durchaus gescheit - nur könnte er für gewisse Fälle zu einer unrealistisch hohen Sterberate führen. Dieses Problem beseitigen wir, indem wir die Exposure der STerbenden (und, vermutlich, der emigrierenden) jeweils auf eins setzen. 

## VL2: Smoothing: Mechanical/Whittaker-Henderson
Auch die Idee der zweiten Vorlesung ist einigermaßen rasch erzählt. Im Wesentlichen geht es nun darum, die beobachteten Sterberaten etwas glatter zu gestalten, vermutlich v.a. im Hinblick auf die späteren Ideen von Forecasting. Besonders wild fluktuierende Sterberaten in x oder t scheinen auch wenig sinnvoll. Dafür gibt es verschiedene Methoden, von denen wir insbesondere das Smoothing mit Splines für wichtig genommen haben. 
* Mechanical Smoothing: Das ist die trivialste Methode. In einer Umgebung von i-r bis i+s wird gewichtet über alle beobachteten Sterberaten $\tilde q$ gemittelt. 
* Eine doppelte Mittlung mit jeweils einer Umgebung von $\pm 2$ erfolgt bei der Methode von Finliaison Wittstein mit Gewicht 1/5. Durch die doppelte Mittlung erweitert sich der endgültige Bereich zwar, wird jedoch an den Rändern weniger stark berücksichtigt als in der "Mitte". 
* Whittaker-Henderson hat als Idee, gleichzeitig auf Abweichung (Fitting) als auch auf Glattheit (Smoothing) hin zu optimieren: Dafür führen wir ein Maß der Glattheit (Smoothness) ein, das wir mithilfe des Konzepts der diskreten Ableitung  definieren: $\Delta ^k x_i = \sum_{j=0}^k (-1)^{k-j} \binom{k}{j} x_{i+j} $, $S^d (q) = \sum_{k = 0}^{w-d} \Delta^k q_k$. Eigentlich ist das eher ein Maß für die Un-Smoothness, da wir sagen, dass etwas "glatter" ist wenn $S^d$ kleiner ist. Aber das soll uns hier nicht weiter stören. Nun brauchen wir also noch die Abweichung, die trivial definiert ist: Für $q$ die geglättete und $\tilde q$ die beobachtete Sterberate definieren wir als Abweichung $\sum_{k=0}^w w_k (q_k - \tilde q_k)^2$. Nun gilt es also, das ganze zu minimieren, indem wir die Summe der beiden minimieren und dabei noch einen Grad $g$ definieren, mit dem wir die Smoothness betrachten. In der Praxis ist d meist zwischen 2 und 3 herum gewählt. 
Wir lösen das mathematische Minimierungsproblem durch eine Transformation des Problems in eine Matrixgleichung und optimieren analytisch nach einem $q$, das wir ja finden wollen. So finden wir ein optimales $q$. Das ganze ist hier auch implementiert.

## VL3: Cubic natural splines
Die Haupt-Smoothing-Geschichte war diejenige mit den kubischen natürlichen Splines. Kubisch, weil wir mit stückweisen Polynomfunktionen höchstens dritten Grades die Partition auf einem Intervall interpolieren, sodass die Gesamtfunktion 2-glatt ist. Im Wesentlichen ist das auch schon die ganze GEschichte. Der Spline wird mit $S$ bezeichnet, die einzelnen Polynome auf der Partition mit $S_k$. Es gibt einen Operator ohne Namen, $\mathcal I f = \sum_{k=0}^w (\tilde q_k - f(x_k)) + \lambda \int_{x_0}^{x_k} f'' (x) ^2\textup{d} x$. Interessanterweise passiert die Geschichte hier nun ohne ein Gewicht bei der Deviation. Wie auch immer definieren wir eine Basis $N_j$ für die Splines und setzen den Koeffizientenvektor $b$, sd. $S = Nb$. Dann können wir abermals mit einer Umformung in eine Matrixgleichung die Splines einfach berechnen, sodass das gesamte Splineproblem minimiert wird. Am Ende berechnen wir eine solche Matrix, sodass wir für die geglätteten $q$ schreiben können $q = S_\lambda \tilde q$.
Eigentlich schon in **VL4**, aber thematisch noch hier  zugehörig ist die optimale Wahl eines Lambda. Dazu bemühen wir eine Cross-Validation und wollen natürlich den Loss dazu minimieren. Wir berechnen also $CV(\lambda) = \sum_{k=0}^w (\tilde q_k - S_\lambda^(k)(x_k))^2$, wobei der Spline hier ohne Nutzung der k-ten Beobachtung berechnet wird. Man kann nun auch zeigen, dass wir das umformen können nach $\sum_{k=0}^w \frac{(\tilde q_k - S_\lambda(x_k))^2}{(1 - S_\lambda (k,k))}.$ Das vereinfacht die Berechnung für beliebige Lambda und natürlich können wir _dann_ auch nach $\lambda$ hin minimieren.

## VL4: (CV/LOOCV), Bias/Variance-Tradeoff
In dieser Vorlesung wurden die Überlegungen zu den kubischen Splines beendet 